In [2]:
import pandas as pd

# Load datasets
movies = pd.read_csv('/Users/akshay/Desktop/dsbda_practical/DSBDALExam DataSets/DSBDALExam DataSets/Movie/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('/Users/akshay/Desktop/dsbda_practical/DSBDALExam DataSets/DSBDALExam DataSets/Movie/ratings_small.csv')
keywords = pd.read_csv('/Users/akshay/Desktop/dsbda_practical/DSBDALExam DataSets/DSBDALExam DataSets/Movie/keywords.csv')
links = pd.read_csv('/Users/akshay/Desktop/dsbda_practical/DSBDALExam DataSets/DSBDALExam DataSets/Movie/links_small.csv')
credits = pd.read_csv('/Users/akshay/Desktop/dsbda_practical/DSBDALExam DataSets/DSBDALExam DataSets/Movie/credits.csv')

# ------------------ m. Create data subsets for different languages ------------------
# Example: English and French subsets
english_movies = movies[movies['original_language'] == 'en']
french_movies = movies[movies['original_language'] == 'fr']

print("English movies subset shape:", english_movies.shape)
print("French movies subset shape:", french_movies.shape)

# ------------------ n. Merge two subsets ------------------
# Merging English and French movies
merged_lang_movies = pd.concat([english_movies, french_movies], ignore_index=True)
print("Merged language subset shape:", merged_lang_movies.shape)

# ------------------ o. Sort Data using customer ratings ------------------
# Merge ratings with links to get tmdbId
ratings_links = pd.merge(ratings, links, on='movieId')
ratings_links['tmdbId'] = pd.to_numeric(ratings_links['tmdbId'], errors='coerce')

# Merge with movies_metadata using tmdbId
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
merged_ratings = pd.merge(ratings_links, movies, left_on='tmdbId', right_on='id')

# Sort by rating
sorted_by_rating = merged_ratings.sort_values(by='rating', ascending=False)
print("Top 5 movies sorted by user ratings:\n", sorted_by_rating[['title', 'rating']].head())

# ------------------ p. Transposing Data ------------------
# Transpose a small sample for readability
transposed_sample = movies[['id', 'title', 'original_language']].head().transpose()
print("Transposed movie data sample:\n", transposed_sample)

# ------------------ q. Melting Data to long format ------------------
# Melting runtime and revenue into one column
melted = pd.melt(movies[['id', 'title', 'runtime', 'revenue']].head(),
                 id_vars=['id', 'title'],
                 value_vars=['runtime', 'revenue'],
                 var_name='attribute',
                 value_name='value')
print("Melted movie data:\n", melted)

# ------------------ r. Casting data to wide format ------------------
# Pivoting back to wide format
wide_format = melted.pivot(index=['id', 'title'], columns='attribute', values='value').reset_index()
print("Wide format restored:\n", wide_format)

English movies subset shape: (32269, 24)
French movies subset shape: (2438, 24)
Merged language subset shape: (34707, 24)
Top 5 movies sorted by user ratings:
                      title  rating
92196            Airplane!     5.0
25178        The Lion King     5.0
71161   The Usual Suspects     5.0
25189  The Hudsucker Proxy     5.0
71162          Taxi Driver     5.0
Transposed movie data sample:
                            0        1                 2                  3  \
id                     862.0   8844.0           15602.0            31357.0   
title              Toy Story  Jumanji  Grumpier Old Men  Waiting to Exhale   
original_language         en       en                en                 en   

                                             4  
id                                     11862.0  
title              Father of the Bride Part II  
original_language                           en  
Melted movie data:
         id                        title attribute        value
0    86